The main idea here is that I want to guess what are the most important keys ahead of time ()

In [1]:
# For boba

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

import torch
print(torch.cuda.device_count())

2


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16, attn_implementation='sdpa', device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

/home/vijay/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/vijay/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
from typing import Optional, Tuple
from transformers.cache_utils import Cache
from transformers.models.llama.modeling_llama import apply_rotary_pos_emb, repeat_kv
import math
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from time import perf_counter

In [7]:
# @torch.compile # probablty shouldn't include this the first few times?
def low_dim_get_attention_scores(q: torch.Tensor, k: torch.Tensor, v: torch.Tensor, block_size: int = 32, dim: int = 32, top_ratio = 4): # then, reduce this again to 8 and 64 after determining stuff and calculate the real deal
    num_tokens = k.shape[-2]
    
    Bq, Hq, Tq, Dq = q.shape
    Bk, Hk, Tk, Dk = k.shape

    print("Queries shape: ", q.shape, " Keys shape: ", k.shape, " Values shape: ", v.shape)
    # Dimension B, H, T, D for everything
    q = q.transpose(-1, -2)
    k = k.transpose(-1, -2)
    v = v.transpose(-1, -2)

    q = q.view(Bq * Hq, Dq, Tq)
    k = k.view(Bk * Hk, Dk, Tk)
    v = v.view(Bk * Hk, Dk, Tk)

    q_chunks = F.avg_pool1d(q, kernel_size=block_size, stride=block_size)
    k_chunks = F.avg_pool1d(k, kernel_size=block_size, stride=block_size) 
    q_chunks = q_chunks.transpose(-1, -2)
    k_chunks = k_chunks.transpose(-1, -2)

    # q_chunks = q_chunks.view(Bk, Hk, Tk // block_size, Dk)
    # k_chunks = q_chunks.view(Bk, Hk, Tk // block_size, Dk) -> reviewing them can happen later

    print("Q chunks shape: ", q_chunks.shape, " K chunks shape: ", k_chunks.shape)

    # Move q, k into lower dimension with random matrices
    random_matrix = torch.randn(Dk, dim, device=q_chunks.device)
    q_ldim = torch.matmul(q_chunks, random_matrix)
    k_ldim = torch.matmul(k_chunks, random_matrix)
    print("Lower dim matrix shapes: ", q_ldim.shape, k_ldim.shape)

    # Attention scores
    approx_scores = F.softmax(torch.bmm(q_ldim, k_ldim.transpose(-1, -2)), dim=-1) # Add causal attention mask
    approx_scores = torch.tril(approx_scores) # TODO: add support for adding an attention mask?
    print("Attention map: ", approx_scores.shape)
    # The keys should be in dim -2

    # chunk_size = 
    num_top_block_count = (num_tokens // block_size // top_ratio)
    chunk_count = num_tokens // block_size
    print("Top block count: ", num_top_block_count)
    top_blocks = torch.sort(torch.topk(approx_scores, num_top_block_count, dim=-1).indices).values
    print("Top blocks shape: ", top_blocks.shape)
    print("Top blocks: ", top_blocks)

    top_blocks = top_blocks.reshape(Bq, Hq, Tq // block_size, num_top_block_count)

    print("Trying to create matrix of size: ", (Bk * chunk_count * Hk * num_top_block_count * block_size * Dk))

    k_blocks = torch.zeros((Bk * chunk_count, Hk, num_top_block_count * block_size, Dk), device=k.device)
    v_blocks = torch.zeros((Bk * chunk_count, Hk, num_top_block_count * block_size, Dk), device=k.device)
    for batch_idx in range(Bk):
        for chunk_idx in range(chunk_count):
            for head_idx in range(Hk):
                for block_index_idx in range(num_top_block_count):
                    block_index = top_blocks[batch_idx, head_idx, chunk_idx, block_index_idx]
                    if block_index_idx > chunk_idx: # keep the lower triangle
                        break
                    k_blocks[batch_idx * chunk_count + chunk_idx, head_idx, block_index_idx * block_size:(block_index_idx + 1) * block_size] = k[batch_idx, head_idx, chunk_idx, (block_index * block_size):((block_index + 1) * block_size)]
                    v_blocks[batch_idx * chunk_count + chunk_idx, head_idx, block_index_idx * block_size:(block_index_idx + 1) * block_size] = v[batch_idx, head_idx, chunk_idx, (block_index * block_size):((block_index + 1) * block_size)]

    repeated_indices = top_blocks.unsqueeze(-1).expand(-1, -1, -1, block_size)
    block_offsets = torch.arange(block_size, device=top_blocks.device).view(1, 1, 1, block_size)
    expanded_top_blocks = repeated_indices * block_size + block_offsets
    print("Expanded top blocks shape: ", expanded_top_blocks.shape)
    print("Expanded top blocks shape: ", expanded_top_blocks)

    expanded_top_blocks = torch.flatten(expanded_top_blocks, start_dim=-2, end_dim=-1)
    expanded_top_blocks = expanded_top_blocks.reshape(Bq, Hq, num_tokens // block_size, num_top_block_count * block_size)
    print("Flattened and reshaped expansion: ", expanded_top_blocks.shape)
    
    q_blocks = q.reshape(Bq, Hq, Tq // block_size, block_size, Dq)
    q_blocks = q_blocks.permute(0, 2, 1, 3, 4).reshape(-1, Hq, block_size, Dq)
    print("New q blocks dimension: ", q_blocks.shape)

    expanded_top_blocks = expanded_top_blocks.unsqueeze(-1).expand(-1, -1, -1, -1, Dq)

    print("Expanded top blocks shape: ", expanded_top_blocks.shape)

    k_blocks = torch.gather(k, dim=-2, index=expanded_top_blocks) # -2 says that this is in the token_length portion
    v_blocks = torch.gather(v, dim=-2, index=expanded_top_blocks)
    print("K blocks initial shape : ", k_blocks.shape, " V blocks initial shape: ", k_blocks.shape)

    return q_blocks, k_blocks, v_blocks

In [8]:
def sdpa_forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_value: Optional[Cache] = None,
    output_attentions: bool = False,
    use_cache: bool = False,
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    # print("Running custom forward function")
    bsz, q_len, _ = hidden_states.size()

    query_states = self.q_proj(hidden_states)
    key_states = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)

    query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

    kv_seq_len = key_states.shape[-2]
    if past_key_value is not None:
        kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
    cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)

    query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

    if past_key_value is not None:
        cache_kwargs = {"sin": sin, "cos": cos}  # Specific to RoPE models
        key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)

    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    if attention_mask is not None:
        if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
            raise ValueError(
                f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
            )

    # SDPA with memory-efficient backend is currently (torch==2.1.2) bugged with non-contiguous inputs with custom attn_mask,
    # Reference: https://github.com/pytorch/pytorch/issues/112577.
    if query_states.device.type == "cuda" and attention_mask is not None:
        query_states = query_states.contiguous()
        key_states = key_states.contiguous()
        value_states = value_states.contiguous()

    # generate weights for which tokens are important or not

    # Apply padding to q and k
    max_block_size = 32
    k_len = key_states.shape[-2]
    pad_len_q = 0 if (q_len % max_block_size) == 0 else (max_block_size - (q_len % max_block_size))
    pad_len_k = 0 if (k_len % max_block_size) == 0 else (max_block_size - (k_len % max_block_size))
    if pad_len_q > 0:
        bq, hq, tq, dq = query_states.shape
        pad_matrix = torch.zeros((bq, hq, pad_len_q, dq), device=query_states.device)
        query_states = torch.cat((query_states, pad_matrix), dim=-2)  # Padding on the last dimension
    if pad_len_k > 0:
        bk, hk, tk, dk = key_states.shape
        pad_matrix = torch.zeros((bk, hk, pad_len_k, dk), device=query_states.device)
        key_states = torch.cat((key_states, pad_matrix), dim=-2)  # Padding on the last dimension
        value_states = torch.cat((value_states, pad_matrix), dim=-2)
    print("Padding length: ", pad_len_q, pad_len_k)
    q_blocks, k_blocks, v_blocks = low_dim_get_attention_scores(query_states, key_states, value_states)

    attn_output = torch.nn.functional.scaled_dot_product_attention(
        q_blocks,
        k_blocks,
        v_blocks,
        attn_mask=attention_mask,
        dropout_p=self.attention_dropout if self.training else 0.0,
        # The q_len > 1 is necessary to match with AttentionMaskConverter.to_causal_4d that does not create a causal mask in case q_len == 1.
        is_causal=self.is_causal and attention_mask is None and q_len > 1,
    )

    # TODO: rejoin the output to fit the desired size.

    attn_output = attn_output.transpose(1, 2).contiguous()
    attn_output = attn_output.reshape(bsz, q_len, self.hidden_size) # q_len is padded to be of the same length for everything

    attn_output = self.o_proj(attn_output)

    return attn_output, None, past_key_value

for i in range(len(model.model.layers)):
    model.model.layers[i].self_attn.forward = sdpa_forward.__get__(model.model.layers[i].self_attn, type(model.model.layers[i].self_attn))

In [9]:
sample_text = open("snapkv_full.txt", "r", encoding="utf-8").read()
encoded_tokens = tokenizer(sample_text, return_tensors="pt")
for key in encoded_tokens:
    encoded_tokens[key] = encoded_tokens[key].cuda()

# find a way to start by chunking this portion

# there shouuld be a mix between the global attention and the attention that only the last few tokens (query) pay

past_key_values = None
outputs = model.generate(**encoded_tokens, output_attentions=True, return_dict_in_generate=True, max_new_tokens=1)

Padding length:  15 15
Queries shape:  torch.Size([1, 32, 17600, 128])  Keys shape:  torch.Size([1, 32, 17600, 128])  Values shape:  torch.Size([1, 32, 17600, 128])
Q chunks shape:  torch.Size([32, 550, 128])  K chunks shape:  torch.Size([32, 550, 128])
Lower dim matrix shapes:  torch.Size([32, 550, 32]) torch.Size([32, 550, 32])
Attention map:  torch.Size([32, 550, 550])
Top block count:  137
Top blocks shape:  torch.Size([32, 550, 137])
Top blocks:  tensor([[[  0,   1,   2,  ..., 134, 135, 136],
         [  0,   1,   2,  ..., 134, 135, 136],
         [  0,   1,   2,  ..., 134, 135, 136],
         ...,
         [  0,   1,   2,  ..., 134, 135, 547],
         [  0,   1,   2,  ..., 134, 135, 548],
         [  0,   1,   2,  ..., 134, 547, 548]],

        [[  0,   1,   2,  ..., 134, 135, 136],
         [  0,   1,   2,  ..., 134, 135, 136],
         [  0,   1,   2,  ..., 134, 135, 136],
         ...,
         [  0,   1,   2,  ..., 324, 325, 547],
         [  0,   1,   2,  ..., 325, 547, 548

OutOfMemoryError: CUDA out of memory. Tried to allocate 36.79 GiB. GPU 0 has a total capacity of 23.69 GiB of which 12.03 GiB is free. Including non-PyTorch memory, this process has 11.65 GiB memory in use. Of the allocated memory 10.31 GiB is allocated by PyTorch, and 1.03 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)